In [245]:
import zipfile
import os
from PIL import Image
import numpy as np
import random
import cv2
from pathlib import Path

**№1 Загрузка данных**

In [246]:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        
        for file_info in zip_ref.infolist():
            if file_info.filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                # Извлекаем класс из имени файла (например, 'class1_image1.jpg' -> 'class1')
                class_name = file_info.filename.split('_')[0]  

                class_folder = os.path.join(extract_to_folder, class_name)
                os.makedirs(class_folder, exist_ok=True)

                zip_ref.extract(file_info, class_folder)
                #print(f"Извлечен файл: {file_info.filename} в {class_folder}")

In [247]:
zip_file_path = '/home/c5/Рабочий стол/С[5]/C2_M2_Задание/Gauge.zip'  
extract_to_folder = '/home/c5/Рабочий стол/С[5]/C5_M2/images'  

unzip_file(zip_file_path, extract_to_folder)

Файл /home/c5/Рабочий стол/С[5]/C2_M2_Задание/Gauge.zip успешно распакован в /home/c5/Рабочий стол/С[5]/C5_M2/images.


In [248]:
unzip_file

<function __main__.unzip_file(zip_file_path, extract_to_folder)>

**№2 Предварительная обработка данных**

Функция для загрузки изображения из исходной дириктории

In [230]:
def load_image(directory_path):
    images = []
    valid_extensions = {'.png', '.jpg', '.jpeg', '.gif', '.bmp'}
    
    for file in os.listdir(directory_path):
        if file.lower().endswith(tuple(valid_extensions)):
            image_path = os.path.join(directory_path, file)
            img = Image.open(image_path)
            images.append(img)
            
    return images

Функция для преобразования изображения до 244x244 пикселей с 3 цветными каналами (RGB)

In [249]:
def resize_image(image):
    
    if image.mode != 'RGB':
        image = image.convert('RGB')
    
    width, height = image.size
    ratio = min(244 / width, 244 / height)
    new_size = (int(width * ratio), int(height * ratio))
    
    resized = image.resize(new_size, Image.Resampling.LANCZOS)
    result = Image.new('RGB', (244, 244))
    offset = ((244 - new_size[0]) // 2, (244 - new_size[1]) // 2)
    result.paste(resized, offset)
    
    return result

Функция для преобразования изображения в оттенки серого:

In [250]:
def convert_to_grayscale(image):

    if image.mode == 'L':
        return image.copy()
    
    r, g, b = image.split()
    gray = Image.merge(
        'L',
        [Image.blend(r, Image.blend(g, b, 0.5), 0.5)]
    )
    return gray

Функция которая будет сохранять изображение в дирикторию preprocessed_images с соответствующими поддирикториями и и тем и же именами

In [251]:
def save_image(image, original_image_path, output_directory='preprocessed_images'):
    
    relative_path = os.path.relpath(original_image_path, start=os.path.dirname(original_image_path))
    save_path = os.path.join(output_directory, relative_path)
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    image.save(save_path)
    print(f"Изображение сохранено: {save_path}")

Теперь соберем всё в одну функцию для предобработки данных

In [252]:
def process_single_image(image_path):

    try:
        # Загрузка изображения
        original_image = Image.open(image_path)
        print(f"Загружено изображение: {image_path}")
        
        # Изменение размера
        resized_image = resize_image(original_image)
        print("Изображение изменено в размере")
        
        # Конвертация в серое
        grayscale_image = convert_to_grayscale(resized_image)
        print("Изображение преобразовано в оттенки серого")
        
        # Сохранение результата
        save_path = save_image(grayscale_image, image_path)
        print(f"Обработка завершена успешно!")
        
        return save_path
        
    except Exception as e:
        print(f"Ошибка при обработке изображения: {str(e)}")
        return None

In [253]:
test_image_path = "/home/c5/Рабочий стол/С[5]/C5_M2/images/Gauge/Gauge_big/train/0-00/gauge_rotated_1_jpg.rf.9d18f0f6f3fd29134a35e4a673a5adc7.jpg"
result = process_single_image(test_image_path)

Загружено изображение: /home/c5/Рабочий стол/С[5]/C5_M2/images/Gauge/Gauge_big/train/0-00/gauge_rotated_1_jpg.rf.9d18f0f6f3fd29134a35e4a673a5adc7.jpg
Изображение изменено в размере
Изображение преобразовано в оттенки серого
Изображение сохранено: preprocessed_images/gauge_rotated_1_jpg.rf.9d18f0f6f3fd29134a35e4a673a5adc7.jpg
Обработка завершена успешно!


In [154]:
result

**№3 Функция проходит по всем изображениям**

**№4 Расширение датасета**

Загружает изображение, находит его центр и выполняет поворот на случайный угол от -15 до +15 градусов.

In [254]:
def rotate_random(image_path):
    img = cv2.imread(image_path)
    (h, w) = img.shape[:2]
    center = (w // 2, h // 2)
    angle = np.random.uniform(-15, 15)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(img, M, (w, h))
    
    base_name = os.path.splitext(image_path)[0]
    corrupted_path = f"{base_name}_corrupted.jpg"
    cv2.imwrite(corrupted_path, rotated)
    return corrupted_path

 Добавили случайный черный прямоугольник на фоточку и сохранили

In [255]:
def add_random_rectangle(image_path):
    img = cv2.imread(image_path)
    height, width = img.shape[:2]
    rect_width = np.random.randint(20, 51)
    x = np.random.randint(0, width - rect_width)
    y = np.random.randint(0, height - rect_height)
    
    result = img.copy()
    cv2.rectangle(result, (x, y), (x + rect_width, y + rect_height), (0, 0, 0), -1)
    
    base_name = os.path.splitext(image_path)[0]
    corrupted_path = f"{base_name}_corrupted.jpg"
    cv2.imwrite(corrupted_path, result)
    return corrupted_path

применяем функции ротат и добавление рандомного прямоугольника ко всем изображениям в папке train,  формируем отдельный датасет с поворотами и прямоугольниками

In [256]:
def process_folder(folder_path):
    """Обработка всех изображений в папке"""
    processed_count = 0
    total_files = len(list(folder_path.glob("*.jpg")))
    
    for image_path in folder_path.glob("*.jpg"):
        try:
            # Применяем обе функции последовательно
            rotated_path = rotate_random(str(image_path))
            final_path = add_random_rectangle(rotated_path)
            
            # Перемещаем результат в целевую директорию
            target_dir = Path("/home/c5/Рабочий стол/С[5]/C5_M2/images/Gauge/Gauge_big/preprocessed_images/train")
            target_dir.mkdir(parents=True, exist_ok=True)
            
            final_filename = Path(final_path).name
            destination = target_dir / final_filename
            os.replace(final_path, destination)
            
            processed_count += 1
            print(f"Обработано {processed_count}/{total_files} файлов в папке {folder_path.name}")
            
        except Exception as e:
            print(f"Ошибка при обработке {image_path}: {str(e)}")

In [257]:
def main():
    # Создаем Path объекты для директорий
    train_dir = Path("/home/c5/Рабочий стол/С[5]/C5_M2/images/Gauge/Gauge_big/train")
    target_dir = Path("/home/c5/Рабочий стол/С[5]/C5_M2/images/Gauge/Gauge_big/preprocessed_images/train")
    
    # Создаем целевую директорию если она не существует
    target_dir.mkdir(parents=True, exist_ok=True)

    # Находим все папки в директории train
    folders = [f for f in train_dir.iterdir() if f.is_dir()]
    
    # Обрабатываем каждую папку последовательно
    for folder in folders:
        print(f"\nОбработка папки {folder.name}")
        process_folder(folder)

if __name__ == "__main__":
    main()


Обработка папки 1-88
Ошибка при обработке /home/c5/Рабочий стол/С[5]/C5_M2/images/Gauge/Gauge_big/train/1-88/gauge_rotated_6_jpg.rf.3b30e10bb8be479fbd408952ff7831b6.jpg: name 'rect_height' is not defined
Ошибка при обработке /home/c5/Рабочий стол/С[5]/C5_M2/images/Gauge/Gauge_big/train/1-88/gauge_rotated_1_jpg.rf.d90f11eb0bc136ceebf22289753130ff.jpg: name 'rect_height' is not defined
Ошибка при обработке /home/c5/Рабочий стол/С[5]/C5_M2/images/Gauge/Gauge_big/train/1-88/gauge_rotated_2_jpg.rf.0bf143f49a634ddab5f7f41c1682f59d.jpg: name 'rect_height' is not defined
Ошибка при обработке /home/c5/Рабочий стол/С[5]/C5_M2/images/Gauge/Gauge_big/train/1-88/gauge_rotated_1_jpg.rf.defae91adb86153ee611cb03d0e6b8ae.jpg: name 'rect_height' is not defined
Ошибка при обработке /home/c5/Рабочий стол/С[5]/C5_M2/images/Gauge/Gauge_big/train/1-88/gauge_rotated_5_jpg.rf.63a9f5d52e4a138b571a137a8195a7ff.jpg: name 'rect_height' is not defined
Ошибка при обработке /home/c5/Рабочий стол/С[5]/C5_M2/images/Ga

KeyboardInterrupt: 